**Step 1: Download ERA5 2-m temperature from CDS website. Update to download 2-m temperature and precip grib files** 

In [1]:
%matplotlib inline
import xarray as xr
import numpy as np 
import matplotlib.pyplot as plt
import os
import pandas as pd
from datetime import datetime, timedelta, date

import dask
import dask.array as dda
import dask.distributed as dd

# rhodium-specific kubernetes cluster configuration
import rhg_compute_tools.kubernetes as rhgk

In [2]:
 # install copernicus API to create client instances
!pip install cdsapi 
import cdsapi 

In [3]:
client, cluster = rhgk.get_standard_cluster(extra_pip_packages='cdsapi')
cluster.scale(10)

In [4]:
client

Client Scheduler: gateway://traefik-impactlab-hub-dask-gateway.impactlab-hub:80/impactlab-hub.e328b535a0254c4cbad02a8c57eda207 Dashboard: /services/dask-gateway/clusters/impactlab-hub.e328b535a0254c4cbad02a8c57eda207/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [29]:
cluster.close()

In [8]:
def retrieve_daily_era5(spec):
    '''
    inputs: year(str), month(str), day(str), list of hours in a day
    output: daily grib file with hourly ERA-5 data
    '''
    
    c, year, month, day, hours = spec #
    
    era5_tmp_dir = '/gcs/impactlab-data/climate/source_data/ERA-5/hourly/precip_grib' # precip_grib, grib_files
    filename = 'total_precip_%s_%s_%s.grib' %(year, month, day) #total_precip, t2m
    filepath = os.path.join(era5_tmp_dir, filename)
    
    # first check to be sure file exists
    if os.path.isfile(filepath):
        pass
    else:
        c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type':'reanalysis',
            'variable':'total_precipitation', #total_precipitation, 2m_temperature
            'year':year,
            'month':month,
            'day':day,
            'time':hours,
            'format':'grib'
        },
        filepath)

In [9]:
era_start = '01-03-1979' 
era_end = '01-03-1979'
# make list of daily datetime indices, this includes leap years 
dt_index_full = pd.date_range(start=era_start, end=era_end, freq='D')

# reformat month/day for the retrieval function 
dt_index_years = dt_index_full.year.astype(str)
dt_index_months = dt_index_full.month.map("{:02}".format)
dt_index_days = dt_index_full.day.map("{:02}".format)
daynum = dt_index_full.dayofyear

# make list of hours for retrieval function -- only need one example day
hours = [hr.strftime("%H:%M") for hr in pd.date_range(start='01-01-1979', end='01-02-1979', freq='H')[:-1]]

In [10]:
c_list = []
for i, day in enumerate(dt_index_days):
    c_list.append(cdsapi.Client())

In [11]:
JOBS = [(c, year, month, day, hours) for c, year, month, day in zip(c_list, 
                                                                    dt_index_years, 
                                                                    dt_index_months, 
                                                                    dt_index_days)]

In [12]:
futures = client.submit(retrieve_daily_era5, JOBS[0])
dd.progress(futures)

VBox()

In [20]:
futures_1 = client.map(retrieve_daily_era5, JOBS[:1500])
dd.progress(futures_1)

VBox()

In [21]:
futures_2 = client.map(retrieve_daily_era5, JOBS[1500:3000])
dd.progress(futures_2)

VBox()

In [22]:
futures_3 = client.map(retrieve_daily_era5, JOBS[3000:4500])
dd.progress(futures_3)

VBox()

In [23]:
futures_4 = client.map(retrieve_daily_era5, JOBS[4500:6000])
dd.progress(futures_4)

VBox()

In [11]:
futures_5 = client.map(retrieve_daily_era5, JOBS[6000:7500])
dd.progress(futures_5)

VBox()

In [ ]:
futures_6 = client.map(retrieve_daily_era5, JOBS[7500:9000])
dd.progress(futures_6)

In [ ]:
futures_7 = client.map(retrieve_daily_era5, JOBS[9000:10500])
dd.progress(futures_7)

In [ ]:
futures_8 = client.map(retrieve_daily_era5, JOBS[10500:12000])
dd.progress(futures_8)

In [ ]:
futures_9 = client.map(retrieve_daily_era5, JOBS[12000:13500])
dd.progress(futures_9)

In [ ]:
futures_10 = client.map(retrieve_daily_era5, JOBS[13500:])
dd.progress(futures_10)

In [1]:
# Check download ouput
! ls /gcs/impactlab-data/climate/source_data/ERA-5/hourly/precip_grib